# Proyecto de predicción con regresión del dataset Global Super Store

## Introducción
Este proyecto se centra en el análisis y predicción de beneficios utilizando técnicas de regresión sobre el dataset Global Super Store. El objetivo principal es desarrollar modelos predictivos que nos ayuden a entender los factores que influyen en el beneficio (Profit) de la tienda.

## Dataset
El dataset Global Super Store contiene información detallada sobre ventas, envíos, productos y clientes de una tienda global. Incluye variables como fechas de pedido y envío, información del cliente, detalles del producto, ventas, descuentos y beneficios.

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Configuración de visualización
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)

# Carga del dataset
df = pd.read_csv('data/Global_Superstore2.csv', encoding='iso-8859-1')

# Mostrar las primeras 10 filas del dataset
print("Dimensiones del dataset:", df.shape)
df.head(10)

## Análisis Exploratorio de Datos (EDA)

### Análisis Inicial
Comenzaremos con un análisis exploratorio para entender mejor nuestros datos. Primero, examinaremos:
1. Tipos de datos y valores faltantes
2. Estadísticas descriptivas básicas
3. Distribución de variables numéricas clave

In [ ]:
# Información general del dataset
print("Información del dataset:")
print(df.info())

print("\nEstadísticas descriptivas de las variables numéricas:")
print(df.describe())

### Análisis de Variables Objetivo y Principales

Analizaremos las variables más importantes para nuestro modelo:
1. Profit (variable objetivo)
2. Sales
3. Discount
4. Category

También examinaremos las correlaciones entre estas variables.

In [ ]:
# Análisis de la variable objetivo (Profit)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(df['Profit'], kde=True)
plt.title('Distribución de Profit')
plt.xlabel('Profit')

plt.subplot(1, 2, 2)
sns.boxplot(y=df['Profit'])
plt.title('Boxplot de Profit')
plt.tight_layout()
plt.show()

# Estadísticas descriptivas de Profit
print("\nEstadísticas de Profit:")
print(df['Profit'].describe())

In [ ]:
# Análisis de relaciones entre variables numéricas principales
plt.figure(figsize=(15, 5))

# Relación entre Sales y Profit
plt.subplot(1, 3, 1)
sns.scatterplot(data=df, x='Sales', y='Profit')
plt.title('Sales vs Profit')

# Relación entre Discount y Profit
plt.subplot(1, 3, 2)
sns.scatterplot(data=df, x='Discount', y='Profit')
plt.title('Discount vs Profit')

# Profit por Category
plt.subplot(1, 3, 3)
sns.boxplot(data=df, x='Category', y='Profit')
plt.title('Profit por Category')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Correlaciones entre variables numéricas
correlation_matrix = df[['Profit', 'Sales', 'Discount', 'Quantity', 'Shipping Cost']].corr()
print("\nMatriz de correlación:")
print(correlation_matrix['Profit'].sort_values(ascending=False))

## Ingeniería de Características

Basándonos en el análisis exploratorio, realizaremos las siguientes transformaciones:

1. Procesamiento de fechas:
   - Crear variable de tiempo de envío
   - Extraer mes y año de la fecha de orden

2. Codificación de variables categóricas:
   - One-hot encoding para Category, Sub-Category, Ship Mode
   - Label encoding para variables con alta cardinalidad como Customer ID

3. Eliminación de columnas redundantes o no informativas

In [ ]:
# Procesamiento de fechas
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

# Crear tiempo de envío (en días)
df['shipping_time'] = (df['Ship Date'] - df['Order Date']).dt.days

# Extraer mes y año
df['order_month'] = df['Order Date'].dt.month
df['order_year'] = df['Order Date'].dt.year

# Seleccionar columnas para one-hot encoding
categorical_columns = ['Category', 'Sub-Category', 'Ship Mode']
onehot = OneHotEncoder(sparse_output=False)

# Aplicar one-hot encoding
onehot_encoded = onehot.fit_transform(df[categorical_columns])
onehot_columns = []
for i, col in enumerate(categorical_columns):
    for j in onehot.categories_[i]:
        onehot_columns.append(f"{col}_{j}")

# Crear DataFrame con variables codificadas
onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_columns)

# Seleccionar columnas numéricas relevantes
numeric_columns = ['Sales', 'Quantity', 'Discount', 'Shipping Cost', 
                  'shipping_time', 'order_month', 'order_year']

# Combinar features numéricas y categóricas
X = pd.concat([df[numeric_columns].reset_index(drop=True), onehot_df], axis=1)
y = df['Profit']

# División de datos (train-test split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensiones de los conjuntos de datos:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

# Guardar los conjuntos de datos
X_train.to_csv('data/X_train.csv', index=False)
X_test.to_csv('data/X_test.csv', index=False)
y_train.to_csv('data/y_train.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)

## Fase 1: Análisis de Línea Base (Modelos Lineales)

En esta fase implementaremos tres modelos de regresión:
1. Regresión Lineal Simple
2. Ridge (L2)
3. Lasso (L1)

Evaluaremos su rendimiento utilizando las métricas R² y RMSE, y analizaremos la importancia de las variables en la predicción.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Cargar datos preprocesados
X_train = pd.read_csv('data/X_train.csv')
X_test = pd.read_csv('data/X_test.csv')
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')

# Convertir y_train y y_test de DataFrame a Series
y_train = y_train.iloc[:, 0]
y_test = y_test.iloc[:, 0]

# Función para evaluar modelos
def evaluate_model(y_true, y_pred, model_name):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"\nResultados para {model_name}:")
    print(f"R² Score: {r2:.4f}")
    print(f"RMSE: {rmse:.2f}")
    return r2, rmse

### 1. Regresión Lineal Simple

Comenzamos con un modelo de regresión lineal simple como línea base para nuestras predicciones.

In [ ]:
# Entrenar modelo de regresión lineal
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predicciones
lr_train_pred = lr_model.predict(X_train)
lr_test_pred = lr_model.predict(X_test)

# Evaluar modelo
print("Evaluación del modelo de Regresión Lineal:")
train_r2, train_rmse = evaluate_model(y_train, lr_train_pred, "Regresión Lineal (train)")
test_r2, test_rmse = evaluate_model(y_test, lr_test_pred, "Regresión Lineal (test)")

### 2. Regularización Ridge (L2)

Implementaremos Ridge con búsqueda de hiperparámetros usando GridSearchCV para encontrar el mejor valor de alpha.

In [ ]:
# Importar GridSearchCV
from sklearn.model_selection import GridSearchCV

# Definir parámetros para búsqueda
alphas = np.logspace(-4, 4, 20)
param_grid = {'alpha': alphas}

# Crear y ajustar GridSearchCV
ridge = Ridge()
grid_ridge = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')
grid_ridge.fit(X_train, y_train)

# Mejor modelo Ridge
best_ridge = grid_ridge.best_estimator_
print(f"Mejor valor de alpha para Ridge: {grid_ridge.best_params_['alpha']:.6f}")

# Predicciones con el mejor modelo Ridge
ridge_train_pred = best_ridge.predict(X_train)
ridge_test_pred = best_ridge.predict(X_test)

# Evaluar modelo
print("\nEvaluación del modelo Ridge:")
ridge_train_r2, ridge_train_rmse = evaluate_model(y_train, ridge_train_pred, "Ridge (Train)")
ridge_test_r2, ridge_test_rmse = evaluate_model(y_test, ridge_test_pred, "Ridge (Test)")

### 3. Regularización Lasso (L1)

Implementaremos Lasso con búsqueda de hiperparámetros. Este modelo nos ayudará a identificar las variables más importantes, ya que puede reducir coeficientes a cero.

In [ ]:
# Crear y ajustar GridSearchCV para Lasso
lasso = Lasso()
grid_lasso = GridSearchCV(lasso, param_grid, cv=5, scoring='r2')
grid_lasso.fit(X_train, y_train)

# Mejor modelo Lasso
best_lasso = grid_lasso.best_estimator_
print(f"Mejor valor de alpha para Lasso: {grid_lasso.best_params_['alpha']:.6f}")

# Predicciones con el mejor modelo Lasso
lasso_train_pred = best_lasso.predict(X_train)
lasso_test_pred = best_lasso.predict(X_test)

# Evaluar modelo
print("\nEvaluación del modelo Lasso:")
lasso_train_r2, lasso_train_rmse = evaluate_model(y_train, lasso_train_pred, "Lasso (Train)")
lasso_test_r2, lasso_test_rmse = evaluate_model(y_test, lasso_test_pred, "Lasso (Test)")

### Análisis de Coeficientes e Interpretación

Analizaremos los coeficientes del modelo Lasso para entender qué variables son las más importantes para predecir el beneficio y cuáles tienen menor impacto o son eliminadas por el modelo.

In [ ]:
# Obtener coeficientes del modelo Lasso
feature_names = X_train.columns
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': best_lasso.coef_
})

# Ordenar coeficientes por valor absoluto
coef_df['Abs_Coefficient'] = abs(coef_df['Coefficient'])
coef_df_sorted = coef_df.sort_values('Abs_Coefficient', ascending=False)

# Mostrar los 10 coeficientes más importantes
print("Top 10 variables más importantes según Lasso:")
print(coef_df_sorted.head(10))

# Contar variables eliminadas (coeficientes = 0)
zero_coef = len(coef_df[coef_df['Coefficient'] == 0])
print(f"\nNúmero de variables eliminadas por Lasso (coeficiente = 0): {zero_coef}")

# Visualizar coeficientes más importantes
plt.figure(figsize=(12, 6))
sns.barplot(data=coef_df_sorted.head(10), x='Coefficient', y='Feature')
plt.title('Top 10 variables más importantes (Lasso)')
plt.xlabel('Coeficiente')
plt.tight_layout()
plt.show()

### Resumen de Resultados

Hemos completado el análisis de línea base utilizando tres modelos diferentes:
1. Regresión Lineal Simple: establece nuestra línea base de rendimiento
2. Ridge: aplica regularización L2 para controlar la complejidad del modelo
3. Lasso: aplica regularización L1 para selección de variables

Los resultados nos muestran:
- El rendimiento base en términos de R² y RMSE
- Las variables más importantes identificadas por Lasso
- Qué variables fueron eliminadas por completo
- La interpretación de los coeficientes en términos de impacto en el beneficio

Esta información nos servirá como punto de partida para futuros análisis y mejoras del modelo.